In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
dfColor = pd.read_csv("../data/clean/Colours.csv")
dfColor.head()

,Name0,Desc1,Desc2,Name1,HEX,R,G,B,Hue,Sat,Lum
0,black,Black,Black,Black,#000000,0,0,0,0.0,0.0,0.0
1,red,medium dark red,Dark red,Deep maroon,#820000,130,0,0,0.0,100.0,25.5
2,red,medium dark red,Dark red,Deep red,#850101,133,1,1,0.0,98.5,26.3
3,red,medium dark red,Dark red,Blood,#8a0303,138,3,3,0.0,95.7,27.6
4,red,medium dark red,Dark red,Darkred,#8B0000,139,0,0,0.0,100.0,27.3


In [9]:
train_set, test_set = train_test_split(dfColor, test_size=0.2, random_state=42)

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin

In [6]:
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

In [3]:
from sklearn import impute

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion

In [8]:
num_attribs = ['Hue', 'Sat', 'Lum']

num_pipeline = Pipeline([
    ('selector', DataFrameSelector(num_attribs)),
    ('imputer', impute.SimpleImputer(strategy="median")),
    ('std_scaler', StandardScaler()),
])

full_pipeline = FeatureUnion(transformer_list=[
    ("num_pipeline", num_pipeline),
])

In [10]:
from sklearn.tree import DecisionTreeClassifier

In [13]:
X = full_pipeline.fit_transform(dfColor)
Y = dfColor['Name0']

In [26]:
tree_clf = DecisionTreeClassifier(max_depth = 4)
tree_clf.fit(X,Y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [15]:
from sklearn.tree import export_graphviz

In [27]:
dotfile = open("col_tree4.dot", 'w')
export_graphviz(tree_clf, out_file=dotfile, feature_names=num_attribs, rounded=True, filled=True, class_names=Y.unique())
dotfile.close()

In [20]:
Y.unique()

array(['black', 'red', 'grey', 'brown', 'white', 'pink', 'orange',
       'yellow', 'green', 'cyan', 'blue', 'purple'], dtype=object)

In [28]:
X

array([[-1.31128787, -2.15405633, -2.62072773],
       [-1.31128787,  1.14162993, -1.34391885],
       [-1.31128787,  1.09219463, -1.3038621 ],
       ...,
       [ 1.51927899,  0.19906366, -1.10858545],
       [ 1.51927899,  1.03287228, -0.85322367],
       [ 1.51927899,  0.27486444, -0.60787608]])